ANÁLISIS EXPLORATORIO COMPLETO - AGRICULTURA URBANA VERTICAL CON FOTOLUMINISCENCIA
==================================================================================

Este script realiza un análisis exploratorio exhaustivo de un sistema de agricultura
urbana vertical en ambiente controlado con tecnología de fotoluminiscencia.
Incluye todas las visualizaciones, matrices de correlación y análisis estadísticos.

Dataset: 50,000 registros de sensores en ambiente controlado cerrado
Variables: 26 parámetros ambientales, lumínicos y de control
Tecnología: Sistema fotoluminiscente para optimización del crecimiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, chi2_contingency
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para visualizaciones
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

Carga y prepara el dataset del sistema de agricultura vertical

In [ ]:
def load_and_prepare_data(file_path):
    """
    Carga y prepara el dataset del sistema de agricultura vertical
    """
    print("SISTEMA DE AGRICULTURA URBANA VERTICAL - ANÁLISIS EXPLORATORIO")
    print("=" * 80)
    print("Cargando datos del ambiente controlado con fotoluminiscencia...")

    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Variables adicionales para análisis temporal
    df['fecha'] = df['timestamp'].dt.date
    df['mes'] = df['timestamp'].dt.month
    df['año'] = df['timestamp'].dt.year
    df['semana'] = df['timestamp'].dt.isocalendar().week

    # Mapeo de días de la semana
    dia_nombres = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves',
                   4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
    df['dia_nombre'] = df['dia_semana'].map(dia_nombres)

    print(f"Dataset cargado exitosamente:")
    print(f"   Registros: {len(df):,}")
    print(f"   Variables: {df.shape[1]}")
    print(f"   Período: {df['timestamp'].min().date()} a {df['timestamp'].max().date()}")
    print(f"   Ambiente: Controlado cerrado con fotoluminiscencia")

    return df

Crea un dashboard general del sistema

In [ ]:
def create_overview_dashboard(df):
    """
    Crea un dashboard general del sistema
    """
    print("\n" + "=" * 80)
    print("1. DASHBOARD GENERAL DEL SISTEMA")
    print("=" * 80)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Dashboard General - Sistema de Agricultura Vertical', fontsize=16, fontweight='bold')

    # 1. Distribución de eficiencia fotosintética
    axes[0,0].hist(df['eficiencia_fotosintetica_pct'], bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0,0].axvline(df['eficiencia_fotosintetica_pct'].mean(), color='red', linestyle='--',
                     label=f'Media: {df["eficiencia_fotosintetica_pct"].mean():.1f}%')
    axes[0,0].set_title('Distribución Eficiencia Fotosintética')
    axes[0,0].set_xlabel('Eficiencia (%)')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # 2. Condiciones del ambiente por categoría
    condiciones_counts = df['condiciones_categoria'].value_counts()
    colors = ['#2E8B57', '#32CD32', '#FF6347']
    axes[0,1].pie(condiciones_counts.values, labels=condiciones_counts.index, autopct='%1.1f%%',
                 colors=colors, startangle=90)
    axes[0,1].set_title('Distribución de Condiciones del Ambiente')

    # 3. Calidad del aire
    calidad_counts = df['calidad_aire_categoria'].value_counts()
    axes[0,2].bar(calidad_counts.index, calidad_counts.values,
                 color=['#FF4444', '#FFA500', '#32CD32', '#006400'])
    axes[0,2].set_title('Calidad del Aire en el Sistema')
    axes[0,2].set_xlabel('Categoría')
    axes[0,2].set_ylabel('Frecuencia')
    axes[0,2].tick_params(axis='x', rotation=45)

    # 4. Temperatura vs Humedad por estación
    seasons_colors = {'Primavera': 'green', 'Verano': 'red', 'Otono': 'orange', 'Invierno': 'blue'}
    for season in df['estacion'].unique():
        season_data = df[df['estacion'] == season]
        axes[1,0].scatter(season_data['temperatura_c'], season_data['humedad_rel_pct'],
                         alpha=0.6, label=season, color=seasons_colors[season], s=1)
    axes[1,0].set_title('Temperatura vs Humedad por Estación')
    axes[1,0].set_xlabel('Temperatura (°C)')
    axes[1,0].set_ylabel('Humedad Relativa (%)')
    axes[1,0].legend()

    # 5. Intensidad fotoluminiscente vs Eficiencia
    axes[1,1].scatter(df['fotoluminiscencia_intensidad'], df['eficiencia_fotosintetica_pct'],
                     alpha=0.5, color='purple', s=1)
    axes[1,1].set_title('Fotoluminiscencia vs Eficiencia Fotosintética')
    axes[1,1].set_xlabel('Intensidad Fotoluminiscente')
    axes[1,1].set_ylabel('Eficiencia Fotosintética (%)')

    # Calcular correlación
    corr, p_value = pearsonr(df['fotoluminiscencia_intensidad'], df['eficiencia_fotosintetica_pct'])
    axes[1,1].text(0.05, 0.95, f'r = {corr:.3f}\np = {p_value:.3e}',
                  transform=axes[1,1].transAxes, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # 6. Purificación requerida por calidad del aire
    purif_by_air = df.groupby('calidad_aire_categoria')['purificacion_requerida_pct'].mean()
    axes[1,2].bar(purif_by_air.index, purif_by_air.values, color='skyblue')
    axes[1,2].set_title('Purificación Requerida por Calidad del Aire')
    axes[1,2].set_xlabel('Calidad del Aire')
    axes[1,2].set_ylabel('Purificación Requerida (%)')
    axes[1,2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # Estadísticas básicas del dashboard
    print("\nESTADÍSTICAS CLAVE DEL SISTEMA:")
    print(f"Eficiencia fotosintética promedio: {df['eficiencia_fotosintetica_pct'].mean():.2f}% ± {df['eficiencia_fotosintetica_pct'].std():.2f}%")
    print(f"Temperatura promedio del ambiente: {df['temperatura_c'].mean():.2f}°C ± {df['temperatura_c'].std():.2f}°C")
    print(f"Humedad relativa promedio: {df['humedad_rel_pct'].mean():.2f}% ± {df['humedad_rel_pct'].std():.2f}%")
    print(f"Intensidad fotoluminiscente promedio: {df['fotoluminiscencia_intensidad'].mean():.2f} ± {df['fotoluminiscencia_intensidad'].std():.2f}")
    print(f"Purificación requerida promedio: {df['purificacion_requerida_pct'].mean():.2f}%")
